# SSAST for speech
All functions copied from the following repo and then edited as indicated

https://github.com/YuanGongND/ssast
Additional authors: Matt, Daniela Wiepert

Original ASTModel class was split into two - one for pretraining and one for finetuning.

The environment must include the following packages, all of which can be dowloaded with pip or conda:
* albumentations
* librosa
* torch, torchvision, torchaudio
* tqdm (this is essentially enumerate(dataloader) except it prints out a nice progress bar for you)
* pyarrow
* timm == '0.4.5'

If running on your local machine and not in a GCP environment, you will also need to install:
* google-cloud-storage

The [requirements.txt](https://github.com/dwiepert/mayo-ssast/blob/main/requirements.txt) can be used to set up this environment. 

To access data stored in GCS on your local machine, you will need to additionally run

```gcloud auth application-default login```

```gcloud auth application-defaul set-quota-project PROJECT_NAME```

In [ ]:
#IMPORTS
#built-in
import argparse
import os
import pickle

#third party
import numpy as np
import pandas as pd
import torch
import pyarrow
from tqdm import tqdm

from google.cloud import storage

#local
from dataloader import AudioDataset
from models import ASTModel_pretrain, ASTModel_finetune
from utilities import *
from loops import *

### Upload/Download functions & Data loading functions
These are defined in `utilities/load_utils.py`.

## Arguments
There are many mutable arguments when running SSAST. See the README for more info on the most important arguments to alter.

In [ ]:
parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
#Inputs
parser.add_argument('-i','--prefix',default='speech_ai/speech_lake/', help='Input directory or location in google cloud storage bucket containing files to load')
parser.add_argument("-s", "--study", choices = ['r01_prelim','speech_poc_freeze_1', None], default='speech_poc_freeze_1', help="specify study name")
parser.add_argument("-d", "--data_split_root", default='gs://ml-e107-phi-shared-aif-us-p/speech_ai/share/data_splits/amr_subject_dedup_594_train_100_test_binarized_v20220620', help="specify file path where datasplit is located. If you give a full file path to classification, an error will be thrown. On the other hand, evaluation and embedding expects a single .csv file.")
parser.add_argument('-l','--label_txt', default='./labels.txt')
parser.add_argument('--lib', default=False, type=bool, help="Specify whether to load using librosa as compared to torch audio")
parser.add_argument("--pretrained_mdl_path", type=str, default='pretrained_model/SSAST-Base-Frame-400.pth', help="the ssl pretrained models path")#, default='/Users/m144443/Documents/mayo_ssast/pretrained_model/SSAST-Base-Frame-400.pth',) #/Users/m144443/Documents/mayo_ssast/pretrained_model/SSAST-Base-Frame-400.pth
parser.add_argument("--finetuned_mdl_path", type=str, default=None, help="if loading an already pre-trained/fine-tuned model")
#GCS
parser.add_argument('-b','--bucket_name', default='ml-e107-phi-shared-aif-us-p', help="google cloud storage bucket name")
parser.add_argument('-p','--project_name', default='ml-mps-aif-afdgpet01-p-6827', help='google cloud platform project name')
parser.add_argument('--cloud', default=False, type=bool, help="Specify whether to save everything to cloud")
#output
parser.add_argument("--dataset", default=None,type=str, help="When saving, the dataset arg is used to set file names. If you do not specify, it will assume the lowest directory from data_split_root")
parser.add_argument("-o", "--exp_dir", default="./experiments", help='specify LOCAL output directory')
parser.add_argument('--cloud_dir', default='', type=str, help="if saving to the cloud, you can specify a specific place to save to in the CLOUD bucket")
#Mode specific
parser.add_argument("-m", "--mode", choices=['train','eval','extraction'], default='train')
parser.add_argument("--task", type=str, default='ft_cls', help="pretraining or fine-tuning task", choices=["ft_avgtok", "ft_cls", "pretrain_mpc", "pretrain_mpg", "pretrain_joint"])
parser.add_argument("--freeze",type=bool, default=True, help="Specify whether to freeze original model before fine-tuning")
parser.add_argument("--weighted",type=bool, default=False, help="Specify whether to train the weight sum of layers")
parser.add_argument("--layer",type=int, default=-1, help="Specify which model layer output to use. Default is -1 which is the final layer.")
parser.add_argument('--embedding_type', type=str, default='ft', help='specify whether embeddings should be extracted from classification head (ft) or base pretrained model (pt)', choices=['ft','pt'])
#Audio configuration parameters
parser.add_argument("--dataset_mean", default=-4.2677393, type=float, help="the dataset mean, used for input normalization")
parser.add_argument("--dataset_std", default=4.5689974, type=float, help="the dataset std, used for input normalization")
parser.add_argument("--target_length", default=1024, type=int, help="the input length in frames")
parser.add_argument("--num_mel_bins", default=128,type=int, help="number of input mel bins")
parser.add_argument("--resample_rate", default=16000,type=int, help='resample rate for audio files')
parser.add_argument("--reduce", default=True, type=bool, help="Specify whether to reduce to monochannel")
parser.add_argument("--clip_length", default=10.0, type=int, help="If truncating audio, specify clip length in seconds. 0 = no truncation")
parser.add_argument("--tshift", default=0, type=float, help="Specify p for time shift transformation")
parser.add_argument("--speed", default=0, type=float, help="Specify p for speed tuning")
parser.add_argument("--gauss", default=0, type=float, help="Specify p for adding gaussian noise")
parser.add_argument("--pshift", default=0, type=float, help="Specify p for pitch shifting")
parser.add_argument("--pshiftn", default=0, type=float, help="Specify number of steps for pitch shifting")
parser.add_argument("--gain", default=0, type=float, help="Specify p for gain")
parser.add_argument("--stretch", default=0, type=float, help="Specify p for audio stretching")
parser.add_argument('--freqm', help='frequency mask max length', type=int, default=0)
parser.add_argument('--timem', help='time mask max length', type=int, default=0)
parser.add_argument("--mixup", type=float, default=0, help="how many (0-1) samples need to be mixup during training")
parser.add_argument("--noise", type=bool, default=False, help="specify if augment noise in finetuning")
parser.add_argument("--skip_norm", type=bool, default=False, help="specify whether to skip normalization on spectrogram")
#Model parameters
parser.add_argument("--fstride", type=int, default=128,help="soft split freq stride, overlap=patch_size-stride")
parser.add_argument("--tstride", type=int, default=2, help="soft split time stride, overlap=patch_size-stride")
parser.add_argument("--fshape", type=int, default=128,help="shape of patch on the frequency dimension")
parser.add_argument("--tshape", type=int, default=2, help="shape of patch on the time dimension")
parser.add_argument('--model_size', default='base',help='the size of AST models', type=str)
#Training parameters
parser.add_argument('--batch_size', default=8, type=int, metavar='N', help='mini-batch size')
parser.add_argument('--num_workers', default=0, type=int, metavar='NW', help='# of workers for dataloading (default: 32)')
parser.add_argument("--epochs", type=int, default=1, help="number of maximum training epochs")
parser.add_argument("--loss", type=str, default="BCE", help="the loss function for finetuning, depend on the task", choices=["BCE", "CE"])
parser.add_argument("--optim", type=str, default="adamw", help="training optimizer", choices=["adamw", "adam"])
parser.add_argument('-lr', '--learning_rate', default=0.001, type=float, metavar='LR', help='initial learning rate')
parser.add_argument("--weight_decay", type=float, default=.0001, help='specify weight decay for adamw')
parser.add_argument("--scheduler", type=str, default=None, help="specify lr scheduler", choices=["onecycle", None])
parser.add_argument("--max_lr", type=float, default=0.01, help="specify max lr for lr scheduler")
#Pretraining parameters
parser.add_argument('--mask_patch', help='how many patches to mask (used only for ssl pretraining)', type=int, default=400)
parser.add_argument("--cluster_factor", type=int, default=3, help="mask clutering factor")
#classification head parameters
parser.add_argument("--activation", type=str, default='relu', help="specify activation function to use for classification head")
parser.add_argument("--final_dropout", type=float, default=0.3, help="specify dropout probability for final dropout layer in classification head")
parser.add_argument("--layernorm", type=bool, default=True, help="specify whether to include the LayerNorm in classification head")
args = parser.parse_args()

## Setting up environment
The first step is to make sure the GCS bucket is initialized if given a `bucket_name`. Additionally, the list of target labels must be set. 

In the original implementation, the list must be given as a `.txt` file to pass through the command line. In this implementation, we will set it as a list.

In [ ]:
# GCS set up
if args.bucket_name is not None:
    storage_client = storage.Client(project=args.project_name)
    bq_client = bigquery.Client(project=args.project_name)
    bucket = storage_client.bucket(args.bucket_name)
else:
    bucket = None

In [ ]:
# (2), check if given study or if the prefix is the full prefix.
if args.study is not None:
    args.prefix = os.path.join(args.prefix, args.study)

In [ ]:
 # (3) get dataset name
if args.dataset is None:
    if args.trained_mdl_path is None or args.mode == 'train':
        if '.csv' in args.data_split_root:
            args.dataset = '{}_{}'.format(os.path.basename(os.path.dirname(args.data_split_root)), os.path.basename(args.data_split_root[:-4]))
        else:
            args.dataset = os.path.basename(args.data_split_root)
    else:
        args.dataset = os.path.basename(args.trained_mdl_path)[:-7]

In [ ]:
#target labels
args.target_labels = ['slow rate',
                        'irregular artic breakdowns',
                        'rapid rate',
                        'distortions',
                        'strained']

#set number of target classes for classification
args.n_class = len(args.target_labels)

In [ ]:
# (5) check if output directory exists, SHOULD NOT BE A GS:// path
if not os.path.exists(args.exp_dir):
    os.makedirs(args.exp_dir)

In [ ]:
# (6) check that clip length has been set
if args.clip_length == 0:
    try: 
        assert args.batch_size == 1, 'Not currently compatible with different length wav files unless batch size has been set to 1'
    except:
        args.batch_size = 1

In [ ]:
# (7) dump arguments
args_path = "%s/args.pkl" % args.exp_dir
with open(args_path, "wb") as f:
    pickle.dump(args, f)
#in case of error, everything is immediately uploaded to the bucket
if args.cloud:
    upload(args.cloud_dir, args_path, bucket)

In [ ]:
  # (8) check if PRETRAINED MDL is stored in gcs bucket
if args.pretrained_mdl_path is None:
    assert args.mode=='train' and 'pretrain' in args.task, 'Must give pretrained model path if not pretraining'
else:
    args.pretrained_mdl_path = gcs_model_exists(args.pretrained_mdl_path, args.bucket_name, args.exp_dir, bucket)


In [ ]:
#(9) add bucket to args
args.bucket = bucket

## Training an SSAST model

We will start with only the finetuning option and not include the option for pretraining or only evaluating an already fine-tuned model (which is available in the full .py script)

The data must be loaded in steps, starting by loading in the label data, then setting up audio configurations for training and evaluation, then generating AudioDataset objects, and finally setting up the dataloaders.

When loading data, we start with a data split root, which we expect to be a directory containing a `train.csv` file and `test.csv` file with file names for train/test and the associated label data.

The audio configurations are dictionaries with parameters for altering the audio and generating spectrograms.

The AudioDatasets are set up in the `dataloader.py` script, using transforms specified in `utilities/speech_utils.py`. 

Finally, the dataloaders take in the datasets and batch size + number of workers.

Please note that the resulting samples will be a dictionary with the keys `uid`, `fbank`, `waveform`, `targets`, `sample_rate`.

In [ ]:
#(1) Load data, note that we are not doing any validation
assert '.csv' not in args.data_split_root, f'May have given a full file path, please confirm this is a directory: {args.data_split_root}'
train_df, val_df, test_df = load_data(args.data_split_root, args.target_labels, args.exp_dir, args.cloud, args.cloud_dir, args.bucket)


In [ ]:
 #(2) set audio configurations (val loader and eval loader will both use the eval_audio_conf
train_audio_conf = {'dataset': args.dataset, 'mode': 'train', 'resample_rate': args.resample_rate, 'reduce': args.reduce, 'clip_length': args.clip_length,
                'tshift':args.tshift, 'speed':args.speed, 'gauss_noise':args.gauss, 'pshift':args.pshift, 'pshiftn':args.pshiftn, 'gain':args.gain, 'stretch': args.stretch,
                'num_mel_bins': args.num_mel_bins, 'target_length': args.target_length, 'freqm': args.freqm, 'timem': args.timem, 'mixup': args.mixup, 'noise':args.noise,
                'mean':args.dataset_mean, 'std':args.dataset_std, 'skip_norm':args.skip_norm}

eval_audio_conf = {'dataset': args.dataset, 'mode': 'evaluation', 'resample_rate': args.resample_rate, 'reduce': args.reduce, 'clip_length': args.clip_length,
                'tshift':args.tshift, 'speed':args.speed, 'gauss_noise':args.gauss, 'pshift':args.pshift, 'pshiftn':args.pshiftn, 'gain':args.gain, 'stretch': args.stretch,
                'num_mel_bins': args.num_mel_bins, 'target_length': args.target_length, 'freqm': args.freqm, 'timem': args.timem, 'mixup': args.mixup, 'noise':args.noise,
                'mean':args.dataset_mean, 'std':args.dataset_std, 'skip_norm':args.skip_norm}

In [ ]:
#(3) Generate audio dataset, note that if bucket not given, it assumes None and loads from local files
train_dataset = AudioDataset(annotations_df=train_df, target_labels=args.target_labels, audio_conf=train_audio_conf, 
                                prefix=args.prefix, bucket=args.bucket, librosa=args.lib) #librosa = True (might need to debug this one)
val_dataset = AudioDataset(annotations_df=val_df, target_labels=args.target_labels, audio_conf=eval_audio_conf, 
                                prefix=args.prefix, bucket=args.bucket, librosa=args.lib) #librosa = True (might need to debug this one)
eval_dataset = AudioDataset(annotations_df=test_df, target_labels=args.target_labels, audio_conf=eval_audio_conf, 
                            prefix=args.prefix, bucket=args.bucket, librosa=args.lib)

In [ ]:
#(4) set up data loaders (val loader always has batchsize 1)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers, collate_fn=collate_fn)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=args.num_workers, pin_memory=True, collate_fn=collate_fn)
eval_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers, collate_fn=collate_fn)


In [ ]:
batch = next(iter(train_loader))

### Spectrograms
We can now visualize a spectrogram

In [ ]:
# get a spectrogram
sample = train_loader.dataset[1]
spectrogram = sample['fbank']
label = sample['targets']

from matplotlib import pyplot as plt
plt.figure(num=1, figsize=(15, 15), dpi=80)
plt.imshow(spectrogram.transpose(1,0).flip(0))
plt.show()

## Set up the model
Set up the model using classes from `ast_models.py`. Since we are not attempting pretraining in this notebook, we assert that 'pretrain' is not the given task

In [ ]:
assert 'pretrain' not in args.task, 'pretraining not supported in this notebook'
#initialize model
ast_mdl = ASTModel_finetune(task=args.task, label_dim=args.n_class, 
                            fshape=args.fshape, tshape=args.tshape, 
                            fstride=args.fstride, tstride=args.tstride,
                            input_fdim=args.num_mel_bins, input_tdim=args.target_length, 
                            model_size=args.model_size, load_pretrained_mdl_path=args.pretrained_mdl_path, 
                            activation=args.activation, final_dropout=args.final_dropout, layernorm=args.layernorm, freeze=args.freeze, 
                            weighted=args.weighted, layer=args.layer)


model_parameters = filter(lambda p: p.requires_grad, ast_mdl.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f'Number of trainable parameters: {params}')


### Run training, evaluation

The model training loops are originally implemented in `loops.py`, but we will include them here for context.

In [ ]:
def validation(model, criterion, dataloader_val):
    '''
    Validation loop for finetuning 
    :param model: SSAST model
    :param criterion: loss function
    :param dataloader_val: dataloader object with validation data
    :return validation_loss: list with validation loss for each batch
    '''
    validation_loss = list()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    with torch.no_grad():
        model.eval()
        for batch in tqdm(dataloader_val):
            x = batch['fbank']
            targets = batch['targets']
            x, targets = x.to(device), targets.to(device)
            o = model(x)
            val_loss = criterion(o, targets)
            validation_loss.append(val_loss.item())

    return validation_loss


In [ ]:
def finetune(model, dataloader_train, dataloader_val = None, 
             optim='adamw', learning_rate=0.001, weight_decay=0.001,
             loss_fn='BCE', sched='onecycle', max_lr=0.01,
             epochs=10, exp_dir='', cloud=False, cloud_dir='', bucket=None):
    """
    Training loop for finetuning SSAST 
    :param model: SSAST model
    :param dataloader_train: dataloader object with training data
    :param dataloader_val: dataloader object with validation data
    :param optim: type of optimizer to initialize
    :param learning_rate: optimizer learning rate
    :param weight_decay: weight decay value for adamw optimizer
    :param loss_fn: type of loss function to initialize
    :param sched: type of scheduler to initialize
    :param max_lr: max learning rate for onecycle scheduler
    :param epochs: number of epochs to run pretraining
    :param exp_dir: output directory on local machine
    :param cloud: boolean indicating whether uploading to cloud
    :param cloud_dir: output directory in google cloud storage bucket
    :param bucket: initialized GCS bucket object
    :return model: finetuned SSAST model
    """
    print('Finetuning start')
    #send to gpu
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    #loss
    if loss_fn == 'MSE':
        criterion = torch.nn.MSELoss()
    elif loss_fn == 'BCE':
        criterion = torch.nn.BCEWithLogitsLoss()
    else:
        raise ValueError(f'Given loss function ({loss_fn}) not supported. Must be either MSE or BCE')
    #optimizer
    if optim == 'adam':
        optimizer = torch.optim.Adam([p for p in model.parameters() if p.requires_grad],lr=learning_rate)
    elif optim == 'adamw':
         optimizer = torch.optim.AdamW([p for p in model.parameters() if p.requires_grad], lr=learning_rate, weight_decay=weight_decay)
    else:
        raise ValueError(f'Given optimizer ({optim}) not supported. Must be either adam or adamw')
    
    if sched == 'onecycle':
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, steps_per_epoch=len(dataloader_train), epochs=epochs)
    else:
        scheduler = None
    
    #train
    for e in range(epochs):
        training_loss = list()
        #t0 = time.time()
        model.train()
        for batch in tqdm(dataloader_train):
            x = batch['fbank']
            targets = batch['targets']
            x, targets = x.to(device), targets.to(device)
            optimizer.zero_grad()
            o = model(x)
            loss = criterion(o, targets)
            loss.backward()
            optimizer.step()
            if scheduler is not None:
                scheduler.step()
            loss_item = loss.item()
            training_loss.append(loss_item)

        if e % 10 == 0:
            #SET UP LOGS
            if scheduler is not None:
                lr = scheduler.get_last_lr()
            else:
                lr = learning_rate
            logs = {'epoch': e, 'optim':optim, 'loss_fn': loss_fn, 'lr': lr, 'scheduler':sched}
    
            logs['training_loss_list'] = training_loss
            training_loss = np.array(training_loss)
            logs['running_loss'] = np.sum(training_loss)
            logs['training_loss'] = np.mean(training_loss)

            print('RUNNING LOSS', e, np.sum(training_loss) )
            print(f'Training loss: {np.mean(training_loss)}')

            if dataloader_val is not None:
                print("Validation start")
                validation_loss = validation(model, criterion, dataloader_val)

                logs['val_loss_list'] = validation_loss
                validation_loss = np.array(validation_loss)
                logs['val_running_loss'] = np.sum(validation_loss)
                logs['val_loss'] = np.mean(validation_loss)
                
                print('RUNNING VALIDATION LOSS',e, np.sum(validation_loss) )
                print(f'Validation loss: {np.mean(validation_loss)}')
            
            #SAVE LOGS
            json_string = json.dumps(logs)
            logs_path = os.path.join(exp_dir, 'logs_ft_epoch{}.json'.format(e))
            with open(logs_path, 'w') as outfile:
                json.dump(json_string, outfile)
            
            #SAVE CURRENT MODEL
            print(f'Saving epoch {e}')
            mdl_path = os.path.join(exp_dir, 'ast_ft_mdl_epoch{}.pt'.format(e))
            torch.save(model.state_dict(), mdl_path)
            
            optim_path = os.path.join(exp_dir, 'ast_ft_optim_epoch{}.pt'.format(e))
            torch.save(optimizer.state_dict(), optim_path)

            if cloud:
                upload(cloud_dir, logs_path, bucket)
                upload(cloud_dir, mdl_path, bucket)
                upload(cloud_dir, optim_path, bucket)

    print('Finetuning finished')
    return model

In [ ]:
def evaluation(model, dataloader_eval):
    """
    Start model evaluation
    :param model: SSAST model
    :param dataloader_eval: dataloader object with evaluation data
    :return preds: model predictions
    :return targets: model targets (actual values)
    """
    print('Evaluation start')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    outputs = []
    t = []
    model = model.to(device)
    with torch.no_grad():
        model.eval()
        for batch in tqdm(dataloader_eval):
            x = batch['fbank']
            x = x.to(device)
            targets = batch['targets']
            targets = targets.to(device)
            o = model(x)
            outputs.append(o)
            t.append(targets)

    outputs = torch.cat(outputs).cpu().detach()
    t = torch.cat(t).cpu().detach()
    
    print('Evaluation finished')
    return outputs, t

In [ ]:
print('Now starting fine-tuning for {:d} epochs'.format(args.epochs))
ast_mdl = finetune(ast_mdl, train_loader, val_loader,
                    args.optim, args.learning_rate, args.weight_decay,
                    args.loss, args.scheduler, args.max_lr, args.epochs,
                    args.exp_dir, args.cloud, args.cloud_dir, args.bucket)

In [ ]:
print('Saving final model')
if ast_mdl.weighted:
    mdl_path = os.path.join(args.exp_dir, '{}_{}_{}_{}_epoch{}_ast_ft_weighted_mdl.pt'.format(args.dataset,args.model_size, args.n_class, args.optim, args.epochs))
else:
    if args.layer==-1:
        args.layer='Final'
    mdl_path = os.path.join(args.exp_dir, '{}_{}_{}_{}_layer{}_epoch{}_ast_ft_mdl.pt'.format(args.dataset,args.model_size, args.n_class, args.optim, args.layer, args.epochs))
torch.save(ast_mdl.state_dict(), mdl_path)

if args.cloud:
    upload(args.cloud_dir, mdl_path, args.bucket)

In [ ]:
#EVALUATE finetuned model
preds, targets = evaluation(ast_mdl, eval_loader)


In [ ]:
if ast_mdl.weighted:
    pred_path = os.path.join(args.exp_dir, '{}_{}_{}_{}_epoch{}_ast_ft_weighted_predictions.pt'.format(args.dataset,args.model_size, args.n_class, args.optim, args.epochs))
    target_path = os.path.join(args.exp_dir, '{}_{}_{}_{}_epoch{}_ast_ft_weighted_targets.pt'.format(args.dataset,args.model_size, args.n_class, args.optim, args.epochs))
else:
    if args.layer==-1:
        args.layer='Final'
    pred_path = os.path.join(args.exp_dir, '{}_{}_{}_{}_layer{}_epoch{}_ast_ft_predictions.pt'.format(args.dataset,args.model_size, args.n_class, args.optim, args.layer, args.epochs))
    targets_path = os.path.join(args.exp_dir, '{}_{}_{}_{}_layer{}_epoch{}_ast_ft_targets.pt'.format(args.dataset,args.model_size, args.n_class, args.optim, args.layer, args.epochs))

torch.save(preds, pred_path)
torch.save(targets, target_path)

if args.cloud:
        upload(args.cloud_dir, pred_path, args.bucket)
        upload(args.cloud_dir, target_path, args.bucket)

## Get Embeddings

Embedding extraction is a slightly different process. We instead load in one csv file, initialize and load a finetuned model, then run the embedding loop which extracts a hidden layer if specified as 'pt', the weighted sum output if specified as 'wt', or the output of the first layer of the classification head 'ft' (which functions as the embedding of dim 768)

In [ ]:
args.data_split_root = 'gs://ml-e107-phi-shared-aif-us-p/speech_ai/share/data_splits/amr_subject_dedup_594_train_100_test_binarized_v20220620/test.csv'
args.embedding_type='ft' #if 'pt', it will get embeddings from only the pretrained model, 'wt' from weighted sum parameter
#args.finetuned_mdl_path = None #TODO: must set to a finetuned model if you want it to load and get embeddings in that way.
args.target_labels = [] #not a required step, but embeddings do not require target labels to run. 
args.n_class = 0

In [ ]:
# Get original model arguments
model_args, args.finetuned_mdl_path = setup_mdl_args(args, args.finetuned_mdl_path)

In [ ]:
# (1) load data to get embeddings for
assert '.csv' in args.data_split_root, f'A csv file is necessary for embedding extraction. Please make sure this is a full file path: {args.data_split_root}'
annotations_df = pd.read_csv(args.data_split_root, index_col = 'uid')

if 'distortions' not in annotations_df.columns:
    annotations_df["distortions"]=((annotations_df["distorted Cs"]+annotations_df["distorted V"])>0).astype(int)
    


In [ ]:
 #(2) set audio configurations
args.mixup=0 #just in case it is missed, args.mixup should always be 0 for embedding extraction
audio_conf = {'dataset': args.dataset, 'mode': 'evaluation', 'resample_rate': args.resample_rate, 'reduce': args.reduce, 'clip_length': args.clip_length,
                'tshift':args.tshift, 'speed':args.speed, 'gauss_noise':args.gauss, 'pshift':args.pshift, 'pshiftn':args.pshiftn, 'gain':args.gain, 'stretch': args.stretch,
                'num_mel_bins': args.num_mel_bins, 'target_length': args.target_length, 'freqm': args.freqm, 'timem': args.timem, 'mixup': args.mixup, 'noise':args.noise,
                'mean':args.dataset_mean, 'std':args.dataset_std, 'skip_norm':args.skip_norm}

# (3) set up dataloader with current args
dataset = AudioDataset(annotations_df=annotations_df, target_labels=args.target_labels, audio_conf=audio_conf, 
                            prefix=args.prefix, bucket=args.bucket, librosa=args.lib)
loader = torch.utils.data.DataLoader(dataset, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers, pin_memory=True, collate_fn=collate_fn) 


In [ ]:
ast_mdl = ASTModel_finetune(task=model_args.task, label_dim=model_args.n_class, 
                            fshape=model_args.fshape, tshape=model_args.tshape, 
                            fstride=model_args.fstride, tstride=model_args.tstride,
                            input_fdim=model_args.num_mel_bins, input_tdim=model_args.target_length, 
                            model_size=model_args.model_size, load_pretrained_mdl_path=model_args.pretrained_mdl_path,
                            activation=model_args.activation, final_dropout=model_args.final_dropout, layernorm=model_args.layernorm, freeze=model_args.freeze,
                            weighted=model_args.weighted, layer=model_args.layer)

if args.finetuned_mdl_path is not None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    sd = torch.load(args.finetuned_mdl_path, map_location=device)
    ast_mdl.load_state_dict(sd, strict=False)
else:
    print(f'Extracting embeddings from only a pretrained model: {args.pretrained_mdl_path}. Extraction method changed to pt.')
    args.embedding_type = 'pt'

Again, the embedding extraction loop is implemented in `loops.py`, but we will include it here for context

In [ ]:
def embedding_extraction(model, dataloader, embedding_type='ft', layer=-1, task='ft_cls'):
    """
    Run a specific subtype of evaluation for getting embeddings.
    :param model: SSAST model
    :param dataloader_eval: dataloader object with data to get embeddings for
    :param embedding_type: string specifying whether embeddings should be extracted from classification head (ft) or base pretrained model (pt)
    :param layer: int indicating which hidden state layer to use.
    :param task: finetuning task, only used for 'pt' or 'wt' embedding extraction.
    :return embeddings: an np array containing the embeddings
    """

    print('Calculating Embeddings')
    embeddings = np.array([])
    # send to gpu
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    with torch.no_grad():
        model.eval()
        for batch in tqdm(dataloader):
            x = batch['fbank']
            x = x.to(device)
            e = model.extract_embedding(x, embedding_type, layer, task)
            if embeddings.size == 0:
                embeddings = e
            else:
                embeddings = np.append(embeddings, e, axis=0)

    return embeddings

Note all the different arguments for the embedding_extraction code. 
You can change the embedding type to:
* `ft`: extracts the embedding from the dense layer of the classification head
* `pt`: extracts the embedding from a hidden state (specified with `layer`)
* `wt`: extracts the embedding after weighted sum of layers

You can also change the pooling mode to one of 'mean', 'sum', 'max'.

The main embedding extraction code is implemented in the `ast_models.py` `ASTModel_finetun` class under `extract_embedding(...)`.

In [ ]:
embeddings = embedding_extraction(ast_mdl, loader, args.embedding_type, args.layer, args.task)


In [ ]:
df_embed = pd.DataFrame([[r] for r in embeddings], columns = ['embedding'], index=annotations_df.index)

### Save embeddings

In [ ]:
if args.embedding_type == 'ft':
    args.layer='NA'
    args.task='NA'
elif args.embedding_type == 'wt':
    args.layer='NA'
elif args.layer==-1:
    args.layer='Final'

try:
    pqt_path = '{}/{}_embedlayer{}_{}_{}_embeddings.pqt'.format(args.exp_dir, args.dataset, args.model_size, args.layer, args.task,args.embedding_type)
    df_embed.to_parquet(path=pqt_path, index=True, engine='pyarrow') 

    if args.cloud:
        upload(args.cloud_dir, pqt_path, args.bucket)
except:
    print('Unable to save as pqt, saving instead as csv')
    csv_path = '{}/{}_embedlayer{}_{}_{}_embeddings.csv'.format(args.exp_dir, args.dataset, args.model_size, args.layer, args.task,args.embedding_type)
    df_embed.to_csv(csv_path, index=True)

    if args.cloud:
        upload(args.cloud_dir, csv_path, args.bucket)

### Visualize AUC curves
This hasn't been tested yet.

In [ ]:
def metrics(args, preds, targets):
    """
    Get AUC scores, doesn't return, just saves the metrics to a csv
    :param args: dict with all the argument values
    :param preds: model predictions
    :param targets: model targets (actual values)
    """
    #get AUC score and all data for ROC curve
    metrics = {}
    pred_mat=torch.sigmoid(preds).numpy()
    target_mat=targets.numpy()
    aucs=roc_auc_score(target_mat, pred_mat, average = None) #TODO: this doesn't work when there is an array with all labels as 0???
    print(aucs)
    data = pd.DataFrame({'Label':args.target_labels, 'AUC':aucs})
    data.to_csv(os.path.join(args.exp_dir, 'aucs.csv'), index=False)
    if args.cloud:
        upload(args.cloud_dir, os.path.join(args.exp_dir, 'aucs.csv'), args.bucket)

In [ ]:
# for i in range(len(preds)):
#     fpr, tpr, _ = roc_curve(metrics[2][:,i],  metrics[1][:,i])
#     plt.plot(fpr,tpr,label=metrics[0]['Label'][i]+", auc="+str(round(metrics[0]['AUC'][i],3)))
#     plt.legend(loc=4)
#     plt.show()